In [1]:
import json
import openai
import os
import random

import numpy as np
import pandas as pd

Import openai api key from environment variable - here I have it stored as `LOCAL_OPENAI_API_KEY` but you can just paste yours in instead

In [2]:

openai.api_key = os.environ.get('LOCAL_OPENAI_API_KEY')

In [3]:
def query(prompt, **kwargs):
  """
  wrapper for the API to easily parse data
  """
  
  args = {
    "engine":"text-davinci-002", # using the original davinci
    "temperature":0, # 0 temperature means it's greedy and gives the same result every time (ish)
    "max_tokens":500, # 500 tokens should be enough
    "stop":"\n\n", # we'll use double newlines to separate the examples
  }
  
  args = {**args, **kwargs}
  
  r = openai.Completion.create(prompt=prompt, **args)["choices"][0]["text"].strip()
  return r


Great, the API key is loaded. Now we can start using the API.

In [4]:
query("q: what is 1+1?\na:")

'2'

This downloads the WIC dataset - you may need to install wget if you don't have it installed https://ports.macports.org/port/wget/

In [10]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-02-26 21:07:56--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 2606:50c0:8001::153, 2606:50c0:8002::153, 2606:50c0:8003::153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|2606:50c0:8001::153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip’

WiC_dataset.zip     100%[===================>] 269.52K  --.-KB/s    in 0.1s    

2023-02-26 21:07:56 (2.71 MB/s) - ‘WiC_dataset.zip’ saved [275984/275984]



In [11]:
import zipfile
with zipfile.ZipFile("WiC_dataset.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [5]:
train = pd.read_csv("train/train.data.txt", sep='\t', header=None)
train.columns = ["target", "pos", "position", "context-1", "context-2"]
train_gold = pd.read_csv("train/train.gold.txt", sep='\t', header=None)
train_gold.columns = ["label"]
train = pd.concat([train_gold,train], axis=1)


In [6]:
train[train.pos=="V"].tail(18)[:5]


,label,target,pos,position,context-1,context-2
5386,F,admit,V,3-1,The French doors admit onto the yard .,He admitted his errors .
5387,F,exhaust,V,0-4,Exhaust one 's savings .,This kind of work exhausts me .
5389,F,kill,V,0-2,Kill the engine .,She was killed in the collision of three cars .
5390,T,admit,V,1-1,To admit a serious thought into the mind .,She admitted us here .
5394,F,write,V,6-1,How many books did Georges Simenon write ?,Please write to me every week .


We can go and grab the last couple verbs of the train set to use as our few shots. I added some annotation for meaning

In [7]:

fewShotVerb = """ interface comparison{
    "Similar": bool, // whether the two meaings of the word are similar or used to mean different things, should be true | false
}

determineWordSimilarSense(word, context1, context2) : comparison =>{
    return ai.compare(word, context1, context2) // return the comparison object for if the sense of thr word in both contexts is similar
}

//This returns the keys inside of double quotes ("KEY") so we can parse with JSON

"""

Likewise we can do it with nouns

In [8]:

fewShotNoun = """ interface comparison{
    "Similar": bool, // whether the two meaings of the word are similar or used to mean different things, should be true | false
}

determineWordSimilarSense(word, context1, context2) : comparison =>{
    return ai.compare(word, context1, context2) // return the comparison object for if the sense of thr word in both contexts is similar
}

//This returns the keys inside of double quotes ("KEY") so we can parse with JSON

"""

In [9]:
def testRow(row):
  pos = row[1]["pos"]
  examples = fewShotNoun if pos == "N" else fewShotVerb

  context = """determineWordSimilarSense("{}", "{}", "{}")
>>>""".format(row[1]["context-1"], row[1]["context-2"], row[1]["target"])
  res = query(examples + context)
  try:
    parsed = json.loads(res)
  except Exception as e:
    print('error parsing: ', e)
    # check if there's a true in the last 100 characters
    parsed = {
      "Context": res,
      "Similar": "true" in res.lower()[-100:]
    }

  return parsed


In [10]:
scores = {'V': 0, 'N': 0}
attempted = {'V': 0, 'N': 0}
for row in train.head(10).iterrows():
    print('scores', scores)
    print('attempted', attempted)
    actual = row[1]["label"]
    output = testRow(row)
    print('output:', output)
    pos = row[1]["pos"]
    attempted[pos] += 1

    if actual == "T":
        if output['Similar']:
            scores[pos] += 1
        
    if actual == "F":
        if not output['Similar']:
            scores[pos] += 1



scores {'V': 0, 'N': 0}
attempted {'V': 0, 'N': 0}
output: {'Similar': True}
scores {'V': 0, 'N': 0}
attempted {'V': 1, 'N': 0}
output: {'Similar': True}
scores {'V': 0, 'N': 0}
attempted {'V': 2, 'N': 0}
output: {'Similar': True}
scores {'V': 0, 'N': 0}
attempted {'V': 3, 'N': 0}
output: {'Similar': True}
scores {'V': 0, 'N': 1}
attempted {'V': 3, 'N': 1}
output: {'Similar': True}
scores {'V': 0, 'N': 1}
attempted {'V': 3, 'N': 2}
output: {'Similar': True}
scores {'V': 0, 'N': 1}
attempted {'V': 4, 'N': 2}
output: {'Similar': True}
scores {'V': 1, 'N': 1}
attempted {'V': 5, 'N': 2}
output: {'Similar': True}
scores {'V': 1, 'N': 1}
attempted {'V': 6, 'N': 2}
output: {'Similar': True}
scores {'V': 2, 'N': 1}
attempted {'V': 7, 'N': 2}
output: {'Similar': True}


In [11]:
scores, attempted

({'V': 2, 'N': 2}, {'V': 7, 'N': 3})

In [12]:
dev = pd.read_csv("dev/dev.data.txt", sep='\t', header=None)
dev.columns = ["target", "pos", "position", "context-1", "context-2"]
dev_gold = pd.read_csv("dev/dev.gold.txt", sep='\t', header=None)
dev_gold.columns = ["label"]
dev = pd.concat([dev_gold,dev], axis=1)


In [13]:
devResults = {}
complete = 0
correct = 0

In [14]:

for row in dev.iterrows():

    if row[0] in devResults:
        continue

    q1 = row[1]["context-1"]
    q2 = row[1]["context-2"]
    target = row[1]["target"]
    actual = row[1]["label"]
    
    pos = row[1]["pos"]

    output = testRow(row)
    
    myResults = {}
    myResults["q1"] = q1
    myResults["q2"] = q2

    myResults["pos"] = row[1]["pos"]

    myResults["target"] = target

    myResults["output"] = output

    myResults["actual"] = actual
    devResults[row[0]] = myResults
    complete +=1
    if actual == "T":
        if output['Similar']:
            correct += 1
    if actual == "F":
        if not output['Similar']:
            correct += 1

    pct = correct/complete
    pct = round(pct, 2)

    print ("Score: {}, Complete: {} Correct: {} Wrong: {}".format(pct, complete, correct, complete-correct))
    with open('text-davinci-002-code-0shot-yolo.json', 'w') as f:
        json.dump(devResults, f)


Score: 0.0, Complete: 1 Correct: 0 Wrong: 1
Score: 0.0, Complete: 2 Correct: 0 Wrong: 2
Score: 0.33, Complete: 3 Correct: 1 Wrong: 2
Score: 0.5, Complete: 4 Correct: 2 Wrong: 2
Score: 0.4, Complete: 5 Correct: 2 Wrong: 3
Score: 0.33, Complete: 6 Correct: 2 Wrong: 4
Score: 0.29, Complete: 7 Correct: 2 Wrong: 5
Score: 0.25, Complete: 8 Correct: 2 Wrong: 6
Score: 0.33, Complete: 9 Correct: 3 Wrong: 6
Score: 0.3, Complete: 10 Correct: 3 Wrong: 7
Score: 0.27, Complete: 11 Correct: 3 Wrong: 8
Score: 0.33, Complete: 12 Correct: 4 Wrong: 8
Score: 0.31, Complete: 13 Correct: 4 Wrong: 9
Score: 0.29, Complete: 14 Correct: 4 Wrong: 10
Score: 0.27, Complete: 15 Correct: 4 Wrong: 11
Score: 0.31, Complete: 16 Correct: 5 Wrong: 11
Score: 0.29, Complete: 17 Correct: 5 Wrong: 12
Score: 0.28, Complete: 18 Correct: 5 Wrong: 13
Score: 0.32, Complete: 19 Correct: 6 Wrong: 13
Score: 0.35, Complete: 20 Correct: 7 Wrong: 13
Score: 0.38, Complete: 21 Correct: 8 Wrong: 13
Score: 0.41, Complete: 22 Correct: 9 Wro

In [15]:
with open('text-davinci-002-code-0shot-yolo.json', 'w') as f:
    json.dump(devResults, f)
